# **Algoritmos de búsqueda**
## **Resolviendo laberintos y escapando de adversarios**


In [ ]:
# código dependiente de la plataforma (Google Colab)

#from google.colab import output;
#output.enable_custom_widget_manager();

In [ ]:
# instalamos el paquete que nos permite trabajar con canvas en los notebooks.
# sólo es necesario en Google Colab y sólo una vez, al inicio de la sesión

#!pip install -q ipycanvas==0.11
#!pip install -q ipyevents


In [15]:
from ipycanvas  import Canvas;

import random;
import math;
import ipycanvas;
import ipyevents;
import time;

from copy import deepcopy;


# **Clase Cell**

Los objetos de esta clase describen una celda en un laberinto.

In [16]:
#===============================================================================
# class Cell 
#-------------------------------------------------------------------------------
"""
Representa una celda en el laberinto
Implanta métodos para:
  constructor y __str__
  propiedades x,y (sólo lectura; posición en el laberinto)
  propiedades north, south, east, west (r/w)
  walls() - retorna los muros (o su ausencia) de la celda.
  isClosed() - True si la celda está rodeada de muros.
"""
#===============================================================================
class Cell:

      #-------------------------------------------------------------------------
      def __init__(self, x, y, v=True):
          self._x=x;
          self._y=y;
          self._walls={ "n":bool(v), "s":bool(v), "e":bool(v), "w":bool(v)};

      #-------------------------------------------------------------------------
      @property
      def x(self): return self._x;

      #-------------------------------------------------------------------------
      @property
      def y(self): return self._y;

      #-------------------------------------------------------------------------
      @property
      def north(self): return (self._walls['n']==True);

      @north.setter
      def north(self,v): self._walls['n']=bool(v);

      #-------------------------------------------------------------------------
      @property
      def south(self): return self._walls['s']==True;

      @south.setter
      def south(self,v): self._walls['s']=bool(v);

      #-------------------------------------------------------------------------
      @property
      def east(self): return self._walls['e']==True;

      @east.setter
      def east(self,v): self._walls['e']=bool(v);

      #-------------------------------------------------------------------------
      @property
      def west(self): return self._walls['w']==True;

      @west.setter
      def west(self,v): self._walls['w']=bool(v);
        
      #-------------------------------------------------------------------------
      def walls(self, d="", v=True):
          if   ( 'N' == d) or ( 'n' == d): self.north=bool(v);
          elif ('!N' == d) or ('!n' == d): self.south=bool(v);
          elif ( 'S' == d) or ( 's' == d): self.south=bool(v);
          elif ('!S' == d) or ('!s' == d): self.north=bool(v);
          elif ( 'E' == d) or ( 'e' == d): self.east =bool(v);
          elif ('!E' == d) or ('!e' == d): self.west =bool(v);
          elif ( 'W' == d) or ( 'w' == d): self.west =bool(v);
          elif ('!W' == d) or ('!w' == d): self.east =bool(v);
          return self._walls;

      #-------------------------------------------------------------------------
      def isClosed(self):
          return all(wall for wall in self._walls.values());
      
      #-------------------------------------------------------------------------
      def distance(self, cell, **kwargs):
          assert type(cell)==Cell;

          if ("method" in kwargs) and (kwargs["method"].lower()=="manhattan"):
             return (abs(self._x-cell._x)+abs(self._y-cell._y));

          if ("method" in kwargs) and (kwargs["method"].lower()=="euclidean"):
             return math.sqrt((self._x-cell._x)**2 + (self._y-cell._y)**2);

          return math.sqrt((self._x-cell._x)**2 + (self._y-cell._y)**2);

      #-------------------------------------------------------------------------
      def __eq__(self, cell):
          if type(cell)!=Cell: return False;
          return (self._x==cell._x) and (self._y==cell._y);

      #-------------------------------------------------------------------------
      def __str__(self):
          return "({},{})[north={}, east={}, south={}, west={}]".format(self._x,self._y,self._walls['n'], self._walls['e'], self._walls['s'], self._walls['w']);

      #-------------------------------------------------------------------------
      def __hash__(self):
          return hash((self._x,self._y));


# **Clase Maze**

Clase que genera y contiene las celdas del laberinto y admite métodos para manejarlas.

In [17]:
#===============================================================================
# class Maze
"""
Clase Laberinto
Implanta lo necesario para trabajar con un laberinto.
"""
#===============================================================================
class Maze:
      #-------------------------------------------------------------------------
      # Método estático que genera un laberinto y devuelve un array 
      # tridimensional con las posibilidades de movimiento en cada celda.
      # En cada celda hay un objeto del tipo Cell.
      # visto en: https://scipython.com/blog/making-a-maze/      
      @staticmethod
      def _createMaze(cols,rows):

          maze=[[Cell(x,y) for y in range(rows)] for x in range(cols)];
        
          def neighbours(current):
              DELTA=[('n',(0,-1)), ('s',(0,+1)), ('e',(+1,0)), ('w',(-1,0))];
              neighs=[];
              for direction,(dx, dy) in DELTA:
                  x2, y2 = current.x+dx, current.y+dy;
                  if (x2 in range(cols)) and (y2 in range(rows)):
                     cell=maze[x2][y2];
                     if cell.isClosed(): neighs.append((direction,cell));
              return neighs;

          n=cols*rows; 
          stack=[];
          current=maze[0][0];
          nv=1;
          while nv < n:
                neighs = neighbours(current);
                if not neighs: 
                   current=stack.pop();
                   continue;
                direction,next = random.choice(neighs);
                current.walls(    direction, False);
                next.   walls("!"+direction, False);               
                stack.append(current);
                current=next;
                nv+=1;

          return maze;

      #-------------------------------------------------------------------------
      def __init__(self, cols, rows):
          self._cols=cols;
          self._rows=rows;
          self._maze=Maze._createMaze(self._cols, self._rows);
            
      #-------------------------------------------------------------------------
      def openHole(self, col, row, d):
          current=self._maze[col][row];
          DELTA=[('N',(0,-1)), ('S',(0,+1)), ('E',(+1,0)), ('W',(-1,0))];          
          for direction,(dx, dy) in DELTA:
              x2, y2 = current.x+dx, current.y+dy;
              if direction==d.upper() and (x2 in range(self._cols)) and (y2 in range(self._rows)):
                  cell=self._maze[x2][y2];
                  current.walls(    direction,False);
                  cell.   walls("!"+direction,False);

      #-------------------------------------------------------------------------
      def openRow(self, row):
          for i in range(self._cols):
              self.openHole(i,row,"E");

      #-------------------------------------------------------------------------
      def openCol(self, col):
          for j in range(self._rows):
              self.openHole(col,j,"S");

      #-------------------------------------------------------------------------
      def options(self, cell, exception=None):
          DELTA=[('n',(0,-1)), ('s',(0,+1)), ('e',(+1,0)), ('w',(-1,0))];
          rt=[];
          for d,(dx, dy) in DELTA:
              x2, y2 = cell.x+dx, cell.y+dy;
              if (x2 in range(self._cols)) and (y2 in range(self._rows)):
                 ncell=Cell(x2,y2);
                 if exception!=None and type(exception)==Cell and exception==ncell:   continue;
                 if exception!=None and type(exception)==list and ncell in exception: continue;
                 if (d=='n') and cell.north==False: rt.append(self._maze[x2][y2]);
                 if (d=='s') and cell.south==False: rt.append(self._maze[x2][y2]);
                 if (d=='e') and cell.east ==False: rt.append(self._maze[x2][y2]);
                 if (d=='w') and cell.west ==False: rt.append(self._maze[x2][y2]);
          return rt;

      #-------------------------------------------------------------------------
      # función para, mediante un canvas, imprimir el laberinto.
      # usaremos una impresión por capas.
      def render(self,canvas,**kwargs):
          with ipycanvas.hold_canvas(canvas):
               width=canvas.width;
               height=canvas.height;

               canvas.clear();
               canvas.line_width=1;
               canvas.fill_style="#885555";
               canvas.stroke_rect(0,0,width-1,height-1);

               ix=width /float(self._cols);
               iy=height/float(self._rows);

               if ("maze" in kwargs) and kwargs["maze"]:
                  for j in range(self._rows):
                        for i in range(self._cols):
                          cell=self._maze[i][j];
                          if cell.north: canvas.stroke_line((i+0)*ix,(j+0)*iy,(i+1)*ix,(j+0)*iy);
                          if cell.south: canvas.stroke_line((i+0)*ix,(j+1)*iy,(i+1)*ix,(j+1)*iy);
                          if cell.east:  canvas.stroke_line((i+1)*ix,(j+0)*iy,(i+1)*ix,(j+1)*iy);
                          if cell.west:  canvas.stroke_line((i+0)*ix,(j+0)*iy,(i+0)*ix,(j+1)*iy);
               
               def print_cell(data, p=0):

                   if type(data)==tuple and len(data)==2: # (tuple|Cell,color)
                      color=data[1];
                      data =data[0];

                      if type(data)==tuple and len(data)==2: # (x,y)
                         x=data[0];
                         y=data[1];

                      if type(data)==Cell: # Cell
                         x=data.x;
                         y=data.y;

                   if type(data)==tuple and len(data)==3:
                      x    =data[0];
                      y    =data[1];
                      color=data[2];

                   assert type(x)     in [int, float];   
                   assert type(y)     in [int, float];   
                   assert type(color) in [str];

                   canvas.fill_style=color;
                   canvas.fill_rect(x*ix+p,y*iy+p,ix-2*p,iy-2*p);
               #-- end print_cell ------------------------

               if "path" in kwargs and kwargs["path"]:
                  (path,c)=kwargs["path"];
                  if path:
                     for p in path:
                         print_cell((p,c),4);

               if "start" in kwargs and kwargs["start"]:
                  start=kwargs["start"];
                  print_cell(start);

               if "player" in kwargs and kwargs["player"]:
                  player=kwargs["player"];
                  print_cell(player);

               if "goal" in kwargs and kwargs["goal"]:
                  goal=kwargs["goal"];
                  print_cell(goal);
                
      #-------------------------------------------------------------------------
      def cell(self, *args):
          assert len(args) in [1,2];
          if len(args)==1:
             assert (type(args[0])==tuple or type(args[0])==list) and len(args[0])==2;
             x=args[0][0];
             y=args[0][1];
          else:
             x=args[0];
             y=args[1];
          assert x in range(self._cols);
          assert y in range(self._rows);
          return self._maze[x][y];

      #-------------------------------------------------------------------------
      def __str__(self):
          rt="";
          for j in range(self._rows):
              for i in range(self._cols):
                  rt+=str(self._maze[i][j]);
          return rt;


# **Valores Iniciales**

In [18]:
# Ejecuta de nuevo esta celda para generar un laberinto distinto.

# Alto y ancho del lienzo, en pixels.
WIDTH=800;
HEIGHT=400;

# Alto y ancho del laberinto, en celdas.
COLS=80//2;
ROWS=40//2;

# Principio y final del laberinto.
START=(0,0);
GOAL =(COLS-1, ROWS-1);

# Generamos un nuevo laberinto
maze=Maze(COLS,ROWS);

#maze.openRow(ROWS//2);
#maze.openCol(COLS//2);


# **Estrategia random**

Partiendo de START, tenemos el objetivo de llegar a GOAL.

Primero usaremos una estrategia aleatoria con la única salvedad que, cuando nos movemos una posición, no podemos volver a esta como siguiente movimiento. Para que sea absolutamente aleatorio, quita el parámetro 'last' en la línea 13.

In [ ]:
canvas = Canvas(width=WIDTH, height=HEIGHT)
display(canvas)

current = maze.cell(START)
goal = maze.cell(GOAL)

last=None
maze.render(canvas, maze= True, goal = (GOAL,'red'), player= (current,'green'))
while current!=goal:
    options=maze.options(current)
    
    if len(option)==0:
        current=last
    elif len(options)==1:
        lasr=current
        current=options[0]
    else:
        last=current
        current=random.choice(options)
        
    time.sleep(0.25)
    maze.render(canvas, maze= True, goal = (GOAL,'red'), player= (current,'green'))




# **Estrategia en profundidad**

Búsqueda recursiva en profundidad.



In [ ]:
canvas = Canvas(width=WIDTH, height=HEIGHT)
display(canvas)

current=maze.cell(START)
goal=maze.cell(GOAL)

def mazeDeep(current, last=None)
    
    if current==goal: return True
    else:
        options=maze.options(current,last)
        for option in options:
            return True
        time.sleep(0.10)
    return False

b= mazeDeep(origin)
if b: print('Haencontrado el final!')
else: print('No ha encontrado el final :-(')


# **Búsqueda heurística**

Búsqueda del objetivo (GOAL, esquina inferior derecha). Aquí Dijsktra y A* entregan un resultado muy parecido.

Esto es debido a que en laberinto, muchas veces, el camino óptimo va en sentido contrario a la distancia al objetivo.

In [ ]:
current = maze.cell(START)
goal = maze.cell(GOAL)

def mazeDijsktra(maze, current, target, avoidList=[], depth=0, closed={})
    
    if current in avoidList:
        return (float('inf'),None)
    if current in closed:
        if closed[current] <= depth
            return (float('inf'),None)
    
    closed[current]=depth
    
    if current==target:
        return (depth, [current])
    else:
        md=float('inf'):
        best=None
        options=maze.options(current)
        for option in options:
            r, p = mazeDijsktra(maze,option,target,avoidList, depth+1, closed)
            if p and r<md:
                md=r
                best=p
        if best:
            best.insert(0,current)
            return (md,best)
        else:
            return (float('inf'),None)


d, path = mazeDijsktra(maze, current, goal, [], 0, {})

print(f'Distancia {d}.')

if path:
    canvas=Canvas(with=WIDTH, height=HEIGHT)
    display(canvas)
    for p in path:
        maze.render(canvas, maze=True, goal=(goal,'red'),
                    player=(p,'green'),
                    path=(path,'#e8dd10')
                   )
        time.sleep(0.20)
        